In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import cv2

plt.ion()   # interactive mode

In [2]:
%matplotlib inline

In [3]:
# Data preprocessing (data augmentation and normalisation)

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
# Before proceeding make sure the data is split into train and val.
# In this case the data is split into 125 train and 75 test following the
# hymenoptera dataset split.

In [4]:
data_dir = '../data/tank_data/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                         data_transforms[x])
                 for x in ['train', 'val']}

In [5]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                        shuffle = True, num_workers = 4)
              for x in ['train', 'val']}

In [6]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [7]:
class_names = image_datasets['train'].classes

In [8]:
class_names

['CT100', 'Platina', 'Pulsar', 'V']

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
device

device(type='cuda', index=0)

In [11]:
# Function to train the model and return the model with the best pred wieghts

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-'*10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        print()
        
    time_elapsed = time.time()-since
    print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [12]:
# Function to diplay predictions of a few images

def visualize_model(model, num_images = 6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [17]:
# Finetuning the convnet 

# model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features

# model_ft.fc = nn.Linear(num_ftrs, 2)

# model_ft = model_ft.to(device)

# criterion = nn.CrossEntropyLoss()

# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [18]:
# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                        num_epochs=25)

In [31]:
model = torchvision.models.mobilenet_v2(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.classifier[1] = nn.Linear(1280, 4)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
        
# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

	 classifier.1.weight
	 classifier.1.bias


In [ ]:
model = train_model(model, criterion, optimizer_conv, exp_lr_scheduler,
                      num_epochs=25)

Epoch 0/24
----------
train Loss: 1.1325 Acc: 0.5564
val Loss: 0.3823 Acc: 0.9710

Epoch 1/24
----------
train Loss: 0.7550 Acc: 0.7345
val Loss: 0.2690 Acc: 0.9227

Epoch 2/24
----------
train Loss: 0.5688 Acc: 0.8000
val Loss: 0.1127 Acc: 0.9903

Epoch 3/24
----------
train Loss: 0.4976 Acc: 0.8182
val Loss: 0.1106 Acc: 0.9614

Epoch 4/24
----------
train Loss: 0.4766 Acc: 0.8400
val Loss: 0.1139 Acc: 0.9565

Epoch 5/24
----------
train Loss: 0.4697 Acc: 0.8255
val Loss: 0.0826 Acc: 0.9758

Epoch 6/24
----------
train Loss: 0.4827 Acc: 0.8145
val Loss: 0.0852 Acc: 0.9662

Epoch 7/24
----------
train Loss: 0.4580 Acc: 0.8582
val Loss: 0.1353 Acc: 0.9565

Epoch 8/24
----------
train Loss: 0.3565 Acc: 0.8800
val Loss: 0.1333 Acc: 0.9517

Epoch 9/24
----------
train Loss: 0.3802 Acc: 0.8582
val Loss: 0.1039 Acc: 0.9565

Epoch 10/24
----------
train Loss: 0.4107 Acc: 0.8727
val Loss: 0.0888 Acc: 0.9614

Epoch 11/24
----------
train Loss: 0.3582 Acc: 0.8764
val Loss: 0.0960 Acc: 0.9565

Ep

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
visualize_model(model)

In [ ]:
torch.save(model.state_dict(), "tank_models_mobile.pth")